In [ ]:
import ee
import geemap
import sys,os
sys.path.append(r'D:\09_Code\Gis_Script\GEE_Func')
from S1_distor_dedicated import load_S1collection,S1_CalDistor,DEM_caculator
from S2_filter import merge_s2_collection
from GEE_DataIOTrans import BandTrans,DataTrans,DataIO,Vector_process
Eq_pixels = DataTrans.Eq_pixels
from GEE_CorreterAndFilters import S1Corrector
from GEEMath import get_minmax
import math
from functools import partial
from tqdm import tqdm 


geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()

## 面转线并采用Aoi取交集过滤-->获取4、9邻域-->计算代求点高程、x\y、入射角
（过滤点数不满足要求的ee.List以及点邻域取值不够的数据点）

In [ ]:
import numpy as np
from scipy.optimize import least_squares
from tqdm import tqdm

def Line2Points(feature,region,scale=30):
    '''经度小的会作为start_point，因此Ascedning左到右，Descending左到右'''
    # 从Feature中提取线几何对象
    line_geometry = ee.Feature(feature).geometry().intersection(region, maxError=1)
    
    # 获取线的所有坐标点
    coordinates = line_geometry.coordinates()
    start_point = ee.List(coordinates.get(0)) # 起点坐标
    end_point = ee.List(coordinates.get(-1))  # 终点坐标
    
    # 获取线段的总长度
    length = line_geometry.length()

    # 计算插入点的数量
    num_points = length.divide(scale).floor()  # .subtract(1)

    # 计算每个间隔点的坐标
    def interpolate(i):
        i = ee.Number(i)
        fraction = i.divide(num_points)
        interpolated_lon = ee.Number(start_point.get(0)).add(
            ee.Number(end_point.get(0)).subtract(ee.Number(start_point.get(0))).multiply(fraction))
        interpolated_lat = ee.Number(start_point.get(1)).add(
            ee.Number(end_point.get(1)).subtract(ee.Number(start_point.get(1))).multiply(fraction))
        return ee.Feature(ee.Geometry.Point([interpolated_lon, interpolated_lat]))

    # 使用条件表达式过滤长度为0的线段
    filtered_points = ee.FeatureCollection(ee.Algorithms.If(num_points.gt(0), 
                                                            ee.FeatureCollection(ee.List.sequence(1, num_points).map(interpolate)),
                                                            ee.FeatureCollection([])))

    return filtered_points

def filter_Listlen(item,len=3):
    '''将list长度小于3的过滤，该函数len=3起步判断，由于len=2的时候会将单点判定为正确，故存在误差为了节省计算量，未对此处进行修改'''
    item_list = ee.List(item)
    return ee.Algorithms.If(item_list.size().gte(len), item_list, None)

def get_neighborhood_info(point,image,scale=15,reduceSacle=30,neighborhood_type='4',Pointfilter=False):
    
    point = ee.Geometry.Point(point)
    buffer_region = point.buffer(scale)

    # 生成矩形区域
    region = buffer_region.bounds()
    # 获取矩形的坐标
    coords  = ee.List(region.coordinates().get(0))

    # 提取四个角点坐标
    corner1 = ee.Geometry.Point(ee.List(coords.get(0)))
    corner2 = ee.Geometry.Point(ee.List(coords.get(1)))
    corner3 = ee.Geometry.Point(ee.List(coords.get(2)))
    corner4 = ee.Geometry.Point(ee.List(coords.get(3)))
    
    if neighborhood_type=='4':
        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                                            corner1.coordinates(), corner2.coordinates(), 
                                            corner3.coordinates(), corner4.coordinates()])
    
    elif neighborhood_type=='9': 
        # 创建线段并计算边的中心点
        edge_center1 = ee.Geometry.LineString([corner1.coordinates(), corner2.coordinates()]).centroid()
        edge_center2 = ee.Geometry.LineString([corner2.coordinates(), corner3.coordinates()]).centroid()
        edge_center3 = ee.Geometry.LineString([corner3.coordinates(), corner4.coordinates()]).centroid()
        edge_center4 = ee.Geometry.LineString([corner4.coordinates(), corner1.coordinates()]).centroid()

        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                        corner1.coordinates(), corner2.coordinates(), 
                        corner3.coordinates(), corner4.coordinates(),
                        edge_center1.coordinates(), edge_center2.coordinates(), 
                        edge_center3.coordinates(), edge_center4.coordinates(),
                        point.coordinates()])
    else:
        print('neighborhood_type should be "4" or "9"')

    # 使用MultiPoint几何对象作为geometry参数
    result = image.reduceRegion(
        reducer=ee.Reducer.toList(),  # 转换为列表
        geometry=region_coord,
        scale=reduceSacle,
        maxPixels=1e9)

    # 在结果中添加点坐标
    result = result.set('point_coordinates', point.coordinates())
    if Pointfilter:
        result = ee.Algorithms.If(ee.List(result.get('angle')).size().gte(int(neighborhood_type)), result, None)
    return result 

def weighted_avg_func(neighbor):
    # 使用ee.Number转换index，因为原始的index是ee.ComputedObject
    neighbors_info = ee.Dictionary(neighbor)

    # 将邻域点信息转换为ee.Array
    lon_coords = ee.Array(neighbors_info.get('longitude'))
    lat_coords = ee.Array(neighbors_info.get('latitude'))
    elevations = ee.Array(neighbors_info.get('elevation'))
    x_ = ee.Array(neighbors_info.get('x'))
    y_ = ee.Array(neighbors_info.get('y'))
    angles = ee.Array(neighbors_info.get('angle'))
    point_coords  = ee.List(neighbors_info.get('point_coordinates'))
    
    # 计算距离和权重
    point1_x = ee.Number(point_coords.get(0))
    point1_y = ee.Number(point_coords.get(1))
    distances = lon_coords.subtract(point1_x).pow(2).add(lat_coords.subtract(point1_y).pow(2)).sqrt()
    weights = distances.pow(-1)
    sum_weights = weights.reduce(ee.Reducer.sum(), [0])

    # 计算加权平均高程
    weighted_elevations = elevations.multiply(weights)
    weighted_avg_elevation = weighted_elevations.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])

    # 计算平均角度
    weighted_angles = angles.multiply(weights)
    weighted_avg_angles = weighted_angles.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])
    
    # 计算平均x
    weighted_X = x_.multiply(weights)
    weighted_avg_X = weighted_X.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])
    
    # 计算平均y
    weighted_Y = y_.multiply(weights)
    weighted_avg_Y = weighted_Y.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])

    return ee.Dictionary({'elevation': weighted_avg_elevation,'angle':weighted_avg_angles,
                          'x':weighted_avg_X,'y':weighted_avg_Y,'point_coordinates':neighbors_info.get('point_coordinates')})

def avg_func(neighbor):
    neighbors_info = ee.Dictionary(neighbor)
    elevations = ee.Array(neighbors_info.get('elevation'))
    angles = ee.Array(neighbors_info.get('angle'))
    x_ = ee.Array(neighbors_info.get('x'))
    y_ = ee.Array(neighbors_info.get('y'))
    return ee.Dictionary({'elevation':elevations.reduce(ee.Reducer.mean(),[0]).get([0]) ,
                           'angle':angles.reduce(ee.Reducer.mean(),[0]).get([0]),
                           'x':x_.reduce(ee.Reducer.mean(),[0]).get([0]),
                           'y':y_.reduce(ee.Reducer.mean(),[0]).get([0]),
                           'point_coordinates':neighbors_info.get('point_coordinates')})

def Volum9_func(neighbors_info):
    
    # 修改方程函数以匹配新的模型
    def equation(params, x, y):
        a, b, c, d, e, f = params
        return a*x**2 + b*y**2 + c*x*y + d*x + e*y + f

    # 修改残差函数，去除z作为变量
    def residuals(params, x, y, z_true):
        z_pred = equation(params, x, y)
        return z_pred - z_true
    
    # 根据求得的参数和给定点的经纬度计算高程
    def calculate_z(params, x, y):
        # 根据新模型参数更新，只有6个参数
        a, b, c, d, e, f = params
        # 更新方程以反映新的模型形式
        z_pred = a*x**2 + b*y**2 + c*x*y + d*x + e*y + f
        return z_pred
    

    lon = np.array(neighbors_info['longitude'])
    lat = np.array(neighbors_info['latitude'])
    elevation = np.array(neighbors_info['elevation'])
    angles = np.array(neighbors_info['angle'])
    x_ = np.array(neighbors_info['x'])
    y_ = np.array(neighbors_info['y'])
    lon_pre, lat_pre = np.array(neighbors_info['point_coordinates'])

    # 初始参数猜测
    initial_guess = np.zeros(6)

    # 使用最小二乘法求得的参数,预测高程
    result0 = least_squares(residuals, initial_guess, args=(lon, lat, elevation))
    params0 = result0.x  
    z_pred = calculate_z(params0, lon_pre, lat_pre)

    # 使用最小二乘法求得的参数,预测angle
    result1 = least_squares(residuals, initial_guess, args=(lon, lat, angles))
    params1 = result1.x  
    angle_pred = calculate_z(params1, lon_pre, lat_pre)
    
    # 使用最小二乘法求得的参数,预测x
    result2 = least_squares(residuals, initial_guess, args=(lon, lat, x_))
    params2 = result2.x  
    x_pred = calculate_z(params2, lon_pre, lat_pre)
    
    # 使用最小二乘法求得的参数,预测y
    result3 = least_squares(residuals, initial_guess, args=(lon, lat, y_))
    params3 = result3.x  
    y_pred = calculate_z(params3, lon_pre, lat_pre)
    
    return {'elevation':z_pred,'angle':angle_pred,
            'x':x_pred,'y':y_pred,
            'point_coordinates':[lon_pre, lat_pre]}

def Flat4_func(neighbors_info):

    lon = neighbors_info['longitude']
    lat = neighbors_info['latitude']
    elevation = neighbors_info['elevation']
    angles = np.array(neighbors_info['angle'])
    x_ = np.array(neighbors_info['x'])
    y_ = np.array(neighbors_info['y'])
    lon_pred, lat_pred = np.array(neighbors_info['point_coordinates'])
    
    A = np.vstack([lon, lat, np.ones(len(lon))]).T  # 构建矩阵A

    b = elevation  # 高程向量b
    # 使用最小二乘法解方程Ax = b
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    # x包含拟合参数a, b, c（注意这里不再是1/c）
    a, b, c = x
    z_pred = a * lon_pred + b * lat_pred + c

    b = angles
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    angle_pred = a * lon_pred + b * lat_pred + c
    
    b = x_
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    x_pred = a * lon_pred + b * lat_pred + c
    
    b = y_
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    y_pred = a * lon_pred + b * lat_pred + c

    return {'elevation':z_pred,'angle':angle_pred,'x':x_pred,'y':y_pred,'point_coordinates':[lon_pred, lat_pred]}

def Bilinear_interp_func(neighbors_info):
    lon = np.array(neighbors_info['longitude'])
    lat = np.array(neighbors_info['latitude'])
    elevation = np.array(neighbors_info['elevation'])
    angles = np.array(neighbors_info['angle'])
    x_ = np.array(neighbors_info['x'])
    y_ = np.array(neighbors_info['y'])
    lon_pred, lat_pred = np.array(neighbors_info['point_coordinates'])

    # 确定目标点的四个最近邻点
    distances = np.sqrt((lon - lon_pred)**2 + (lat - lat_pred)**2)
    idx = np.argsort(distances)[:4]
    lon = lon[idx]
    lat = lat[idx]
    elevation = elevation[idx]
    angles = angles[idx]
    x_ = x_[idx]
    y_ = y_[idx]

    # 计算双线性插值
    def bilinear_interp(values):
        # 根据经纬度构造插值矩阵
        matrix = np.array([
            [1, lon[0], lat[0], lon[0] * lat[0]],
            [1, lon[1], lat[1], lon[1] * lat[1]],
            [1, lon[2], lat[2], lon[2] * lat[2]],
            [1, lon[3], lat[3], lon[3] * lat[3]]
        ])
        # 解线性方程组以获得插值系数
        coeffs = np.linalg.solve(matrix, values)
        # 使用插值系数计算预测值
        return coeffs[0] + coeffs[1] * lon_pred + coeffs[2] * lat_pred + coeffs[3] * lon_pred * lat_pred

    z_pred = bilinear_interp(elevation)
    angle_pred = bilinear_interp(angles)
    x_pred = bilinear_interp(x_)
    y_pred = bilinear_interp(y_)

    return {'elevation': z_pred, 'angle': angle_pred, 'x': x_pred, 'y': y_pred, 'point_coordinates': [lon_pred, lat_pred]}

def apply_map_to_list(input_list, func): return input_list.map(func)

def apply_map_to_list_local(input_list, func): return map(func,input_list)

def Main_CalNeighbor(Templist_A,AOI,Prj_scale,Cal_image,Neighbors='4',Elvevation_model='weighted_avg_elevation'):
    Len_Templist_A =  Templist_A.size().getInfo()
    Neighbors = Neighbors # '9'
    Elvevation_model = Elvevation_model #'weighted_avg_elevation' , 'avg_elevation', 'Area_elavation','Volum_elavation','Bilinear_interp'

    All_PointLine = [] 
    # pbar = tqdm(range(Len_Templist_A),ncols=80)
    pbar = range(Len_Templist_A)
    for i in pbar:
        points = Line2Points(Templist_A.get(i),region=AOI,scale=Prj_scale)  
        points = points.geometry().coordinates()
        All_PointLine.append(points)
        
    EE_PointLine = ee.List(All_PointLine)

    # 过滤点数不符合要求的辅助线
    EE_PointLine = EE_PointLine.map(partial(filter_Listlen,len=3)).removeAll([None])

    cal_neighbors = apply_map_to_list(EE_PointLine, 
                                    lambda x: ee.List(x).map(partial(get_neighborhood_info,image=Cal_image,
                                                                    scale = Prj_scale // 2 if Neighbors == '4' else Prj_scale,
                                                                    reduceSacle = Prj_scale,
                                                                    neighborhood_type=Neighbors,
                                                                    Pointfilter=False)).removeAll([None]))

    cal_neighbors = cal_neighbors.map(partial(filter_Listlen,len=3)).removeAll([None])

    if Elvevation_model == 'weighted_avg_elevation':
        Points_WithH_Angle = apply_map_to_list(
                                            cal_neighbors,      
                                            lambda x: ee.List(x).map(weighted_avg_func))
        Points_WithH_Angle = Points_WithH_Angle.getInfo()
    elif Elvevation_model == 'avg_elevation':
        Points_WithH_Angle = apply_map_to_list(
                                            cal_neighbors,
                                            lambda x: ee.List(x).map(avg_func))
        Points_WithH_Angle = Points_WithH_Angle.getInfo()
    elif (Elvevation_model == 'Area_elavation') or (Elvevation_model == 'Volum_elavation') or (Elvevation_model == 'Bilinear_interp'):
        neighbors_info = cal_neighbors.getInfo() # 非常大
        if Elvevation_model == 'Area_elavation':
            Points_WithH_Angle = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Flat4_func,x))))
        elif Elvevation_model == 'Volum_elavation':
            Points_WithH_Angle = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Volum9_func,x))))
        elif Elvevation_model == 'Bilinear_interp':
            Points_WithH_Angle = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Bilinear_interp_func,x))))
    return Points_WithH_Angle

# 计算n阶导数、曲率-->构建局部极大值索引-->根据n阶导数计算局部极大值点、曲率计算凸性点-->根据波峰波谷过滤局部极大值点

In [ ]:
import numpy as np
from scipy.signal import find_peaks, savgol_filter
from scipy.interpolate import CubicSpline

def compute_derivative_same_length(elev_list,nu=1):
    # 创建一个 CubicSpline 对象
    spline = CubicSpline(range(len(elev_list)), elev_list)
    # 计算导数
    derivative = spline.derivative(nu=nu)(range(len(elev_list)))
    return derivative

def compute_curvature(first_deriv, second_deriv):
    curvature = np.abs(second_deriv) / (1 + first_deriv**2)**1.5
    return curvature

def find_local_extrema_indices(array):
    maxima_indices, _ = find_peaks(array)
    minima_indices, _ = find_peaks(-array)
    return maxima_indices, minima_indices

def filter_local_maxima_by_elevation_difference(local_maxima_indices, local_minima_indices, elevation_list, elevation_difference_threshold=15):
    filtered_maxima_indices = []
    for maxima_index in local_maxima_indices:
        maxima_elevation = elevation_list[maxima_index]
        
        minima_before = local_minima_indices[local_minima_indices < maxima_index]
        minima_after = local_minima_indices[local_minima_indices > maxima_index]
        
        if minima_before.size > 0 and minima_after.size > 0:
            nearest_minima_before = minima_before[-1]
            nearest_minima_after = minima_after[0]
            
            if (np.abs(elevation_list[nearest_minima_before] - maxima_elevation) >= elevation_difference_threshold or
                    np.abs(elevation_list[nearest_minima_after] - maxima_elevation) >= elevation_difference_threshold):
                filtered_maxima_indices.append(maxima_index)
        else:
            filtered_maxima_indices.append(maxima_index)
    
    return np.array(filtered_maxima_indices)

def calculate_extreme_points_for_segment(elevations, elevation_difference_threshold=15):
    elevations = np.array(elevations)

    first_derivative = compute_derivative_same_length(elevations,nu=1)
    second_derivative = compute_derivative_same_length(elevations,nu=2)
    curvature = compute_curvature(first_derivative, second_derivative)
    
    # 计算局部极大值和局部极小值的索引
    elevations_local_maxima_indices, elevations_local_minima_indices = find_local_extrema_indices(elevations)
    curvature_local_maxima_indices,_ =  find_local_extrema_indices(curvature)
    
    # 过滤掉微小起伏波动产生的局部极大值点
    filtered_elevations_maxima_indices = filter_local_maxima_by_elevation_difference(
        elevations_local_maxima_indices, elevations_local_minima_indices, elevations, elevation_difference_threshold)

    # 选择凸性点（与曲率最大点求交集）
    filtered_derivatives_indices = np.where((first_derivative >= 0) & (second_derivative <= 0) & (curvature>=0))
    intersection_indices = np.intersect1d(filtered_derivatives_indices,curvature_local_maxima_indices)

    # 求局部极大值点与凸性点并集
    unionFilter_indices = np.sort(np.unique(np.concatenate((intersection_indices, filtered_elevations_maxima_indices)))).astype(np.uint8)

    return unionFilter_indices,intersection_indices,elevations_local_maxima_indices,filtered_elevations_maxima_indices

# # Calculate extreme points for all segments using map
# elevations_ = [[point['elevation'] for point in segment] for segment in Points_WithH_Angle]
# A = list(map(lambda segment: calculate_extreme_points_for_segment(segment, elevation_difference_threshold=0), elevations_))
# unionFilter_indices = [each[0] for each in A]
# intersection_indices = [each[1] for each in A]
# elevations_local_maxima_indices = [each[2] for each in A]
# filtered_elevations_maxima_indices = [each[3] for each in A]

def Main_CalMaxPoints(Points_WithH_Angle,Orbit):
    # 直接使用所有点
    unionFilter_indices = [np.arange(len(each)).astype(np.uint8) for each in Points_WithH_Angle]

    if Orbit == 'DESCENDING':
        updated_data = [(points[::-1], len(points) - indices - 1) for points, indices in zip(Points_WithH_Angle, unionFilter_indices)]
        Points_WithH_Angle, unionFilter_indices = zip(*updated_data)
    return Points_WithH_Angle,unionFilter_indices

# 根据已有选点计算Leftlayover、RigthLayover、Shadow

1、根据union_indices 在 Points_WithH_Angle上确定极值点
2、根据极值点与其余点交互计算Leftlayover、RigthLayover、Shadow（注意升降轨判定方位）
3、将计算点转为栅格，并导出
4、整合代码，形成完整实例

In [ ]:
from scipy.spatial import distance
from collections import defaultdict

# 定义计算两点之间距离的函数
def calculate_distance(point1, point2, scale=10):
    return distance.euclidean(
        (point1['x'] * scale, point1['y'] * scale), 
        (point2['x'] * scale, point2['y'] * scale))

# 定义计算两点之间角度的函数
def calculate_angle(elevation_difference, distance):
    if elevation_difference > 0:
        return np.arctan2(elevation_difference,distance) * 180 / np.pi
    else: return -1

# 定义判断畸变类型的函数
def check_distortion(arc_angle, reference_angle):
    if arc_angle >= reference_angle: return 1
    else: return 0

# 计算左侧畸变
def calculate_left_layover(points, extreme_point, scale=10):
    features_left = []
    for point in points:
        elevation_difference = extreme_point['elevation'] - point['elevation']
        dist = calculate_distance(extreme_point, point, scale=scale)
        arc_angle = calculate_angle(elevation_difference, dist)
        distortion = check_distortion(arc_angle, extreme_point['angle'])
        if (distortion == 1) & (elevation_difference > 0):
            features_left.append({
                'elevation_difference': elevation_difference,
                'distance': dist,
                'arc_angle': arc_angle,
                'distortion': distortion,
                'distortion_type': 'Leftlayover',
                'first_derivative':0,
                'point_coordinates': point['point_coordinates'],
                'values': 1
            })
    return features_left

# 计算右侧畸变
def calculate_right_layover(features_left, extreme_point, points, scale=10):
    # 筛选出存在左侧畸变的特征
    distorted_features_left = [f for f in features_left if f['distortion'] == 1]

    # 如果存在左侧畸变，继续计算右侧畸变
    if distorted_features_left:
        # 获取距离最远的特征
        max_distance_feature = max(distorted_features_left, key=lambda x: x['distance'])
        max_distance_elevation = max_distance_feature['elevation_difference']

        # 创建一个新的极值点
        new_extreme_point = {
            'elevation': extreme_point['elevation'] - max_distance_elevation,
            'angle': extreme_point['angle'],
            'x':extreme_point['x'],
            'y':extreme_point['y'],
            'point_coordinates': extreme_point['point_coordinates']}

        # 计算右侧畸变
        features_right = []
        for point in points:
            elevation_difference = point['elevation'] - new_extreme_point['elevation']
            elevation_difference2 = extreme_point['elevation'] - point['elevation']
            dist = calculate_distance(new_extreme_point, point, scale=scale)
            if elevation_difference > 0 and elevation_difference2 > 0:
                arc_angle = calculate_angle(elevation_difference, dist)
            else:
                arc_angle = -1
            distortion = check_distortion(arc_angle, extreme_point['angle'])
            if (distortion == 1) & (elevation_difference > 0):
                features_right.append({
                    'elevation_difference': elevation_difference,
                    'distance': dist,
                    'arc_angle': arc_angle,
                    'distortion': distortion,
                    'distortion_type': 'Rightlayover',
                    'first_derivative':0,
                    'point_coordinates': point['point_coordinates'],
                    'values': 5
                })

        # 判断是否存在右侧畸变的特征将极值点追加进去
        if any(f['distortion'] == 1 for f in features_right):
            
            features_right.append({
                'elevation_difference': 0,
                'elevation_difference2': 0,
                'distance': 0,
                'arc_angle': 0,
                'distortion': 1,
                'distortion_type': 'Rightlayover',
                'first_derivative':0,
                'point_coordinates': extreme_point['point_coordinates'],
                'values': 5
            })
        
        return features_right
    else: return []

# 计算阴影畸变
def calculate_shadow(points, extreme_point, scale=10):
    shadow_features = []
    for point in points:
        elevation_difference = extreme_point['elevation'] - point['elevation']
        dist = calculate_distance(extreme_point, point, scale=scale)
        arc_angle = calculate_angle(elevation_difference, dist)
        distortion = check_distortion(arc_angle, 90-extreme_point['angle'])
        if (distortion == 1) & (elevation_difference > 0):
            shadow_features.append({
                'elevation_difference': elevation_difference,
                'distance': dist,
                'arc_angle': arc_angle,
                'distortion': distortion,
                'distortion_type': 'Shadow',
                'first_derivative':0,
                'point_coordinates': point['point_coordinates'],
                'values': 7
            })
    return shadow_features

def calculate_forshort(points):
    '''使用一阶导数判断'''
    elevations = np.array([each['elevation'] for each in points])
    first_derivative = compute_derivative_same_length(elevations,nu=1)
    derivative_indices = np.where(first_derivative>0)
    filtered_points = [points[i] for i in derivative_indices[0]]
    filtered_first_derivative = [first_derivative[i] for i in derivative_indices[0]]

    for i in range(len(filtered_points)):
        filtered_points[i] = {
            'elevation_difference': 999,
            'distortion': 1,  
            'distortion_type': 'Foreshortening',
            'point_coordinates': filtered_points[i]['point_coordinates'],
            'first_derivative':filtered_first_derivative[i],
            'values': 9
        }
    return filtered_points

# 计算畸变特征
def calculate_distortion_features(points, indices, candidate_distortion_points=10, DistanceScale=10):
    distortion_features = []
    for index in indices:
        extreme_point = points[index]

        # 获取左侧和右侧的候选点
        start_index_left = max(0, index - candidate_distortion_points)
        previous_points_left = points[start_index_left:index]

        start_index_right = min(len(points) - 1, index + candidate_distortion_points)
        after_points_right = points[index + 1:start_index_right + 1][::-1]

        # 计算左侧畸变
        features_left = calculate_left_layover(previous_points_left, extreme_point, DistanceScale)

        # 计算右侧畸变
        features_right = calculate_right_layover(features_left, extreme_point, after_points_right, DistanceScale)

        # 计算阴影畸变
        shadow_features = calculate_shadow(after_points_right, extreme_point, DistanceScale)

        # 计算透视收缩
        forshort_features = calculate_forshort(points)

        # 将所有畸变特征合并
        distortion_features.extend(features_left + features_right + shadow_features + forshort_features)

    return distortion_features

# 根据畸变类型分离特征并创建 ee.FeatureCollection
def create_feature_collection(distortion_features):
    features = [
        ee.Feature(
            ee.Geometry.Point(feature['point_coordinates']),
            {'values': feature['values'],
             'distortion_type':feature['distortion_type'],
             'first_derivative_max':feature['first_derivative_max']}  # 保留 value 属性
        )
        for feature in distortion_features
    ]
    return ee.FeatureCollection(features)

def Main_CalDistortion(Points_WithH_Angle, unionFilter_indices):
    processed_data = defaultdict(lambda: {
        'distortion_types': set(), 
        'total_value': 0, 
        'first_derivative_max': 0 
    })

    all_distortion_Points = []
    for points, indices in zip(Points_WithH_Angle, unionFilter_indices):
        all_distortion_Points.extend(calculate_distortion_features(points, indices, candidate_distortion_points=10, DistanceScale=10))# DistanceScale:x\y距离度量

    for feature in all_distortion_Points:
        coordinates = tuple(feature['point_coordinates'])
        distortion_type = feature['distortion_type']
        value = feature['values']
        first_derivative = feature['first_derivative']  # 假设每个特征都有 first_derivative 属性

        # 如果这个distortion_type还没有被处理过，将其添加到distortion_types集合中，并更新values
        if distortion_type not in processed_data[coordinates]['distortion_types']:
            processed_data[coordinates]['distortion_types'].add(distortion_type)
            processed_data[coordinates]['total_value'] += value

        # 更新 first_derivative_max
        if first_derivative > processed_data[coordinates]['first_derivative_max']:
            processed_data[coordinates]['first_derivative_max'] = first_derivative

    # 将处理后的数据转换回原始格式
    unique_features = []
    for coordinates, data in processed_data.items():
        unique_features.append({
            'point_coordinates': list(coordinates),
            'distortion_type': '-'.join(sorted(data['distortion_types'])),
            'values': data['total_value'],
            'first_derivative_max': data['first_derivative_max']
        })

    distortion_features = create_feature_collection(unique_features)
    return distortion_features

# 基础数据导入与运算

In [ ]:
import traceback
from tqdm import tqdm
from PackageDeepLearn.utils import DataIOTrans
from IPython.display import clear_output

# 加载藏东南矢量
Southest_doom = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/ChinaShp/SouthestRegion')
Southest_doom_fishnet = geemap.fishnet(Southest_doom.first().geometry(), rows=100, cols=150, delta=1)
lenfish_net = Southest_doom_fishnet.size().getInfo()
Southest_doom_fishnet = Southest_doom_fishnet.toList(lenfish_net)

# 高亚洲渔网
# HighMountainAsia_fishnet = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/HighMountainAsia_fishnet')
# lenfish_net = HighMountainAsia_fishnet.size().getInfo()
# HighMountainAsia_fishnet = HighMountainAsia_fishnet.toList(lenfish_net)

# 设置参数
year = '2019'
START_DATE  = ee.Date(year + '-01-01')
END_DATE   = ee.Date(year + '-12-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
Origin_scale = 10
Prj_scale = 30
Savpath = DataIOTrans.make_dir(r'D:\藏东南几何畸变数据存储')
os.chdir(Savpath)

# 设置DEM
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)
DEM = DEMNASA
Wrong_dataIndex = []

pbar= tqdm(range(lenfish_net), ncols=80)
index_p = 8229
# pbar.update(index_p)

for i in pbar:
    if i < index_p:
        continue
    if i % 100 == 0 and i != 0:
        clear_output(wait=True)
    pbar.set_description('Processing '+str(i))
    AOI = ee.Feature(Southest_doom_fishnet.get(i)).geometry()
    s1_ascending, s1_descending = load_S1collection(AOI,START_DATE,END_DATE,MIDDLE_DATE,FilterSize=30)
    for Orbit in ['ASCENDING', 'DESCENDING']:
        if Orbit == 'ASCENDING':
            S1_image = s1_ascending
        elif Orbit == 'DESCENDING':
            S1_image = s1_descending

        Projection = S1_image.select(0).projection()
        Mask = S1_image.select(0).mask()
        # 获取辅助线
        azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict  = S1Corrector.getS1Corners(S1_image, AOI, Orbit) 
        Heading = azimuthEdge.get('azimuth')
        s1_azimuth_across = ee.Number(Heading).subtract(90.0) # 距离向
        Auxiliarylines = ee.Geometry.LineString([startpoint, endpoint])

        BandTrans.delBands(S1_image, ['VH', 'VV'])
        Cal_image = (Eq_pixels(BandTrans.delBands(S1_image, 'VV','VH').resample('bicubic')).rename('angle')
                            .addBands(ee.Image.pixelCoordinates(Projection))
                            .addBands(DEM.select('elevation'))
                            .addBands(ee.Image.pixelLonLat())
                            .updateMask(Mask)
                            .reproject(crs=Projection, scale=Prj_scale)
                            .clip(AOI))
        try:
            Projection = Cal_image.select(0).projection()
            Templist = S1_CalDistor.AuxiliaryLine2Point(s1_azimuth_across, coordinates_dict, Auxiliarylines,AOI, Prj_scale)
            pbar.set_description('完成Templist')

            Points_WithH_Angle = Main_CalNeighbor(Templist,AOI,Prj_scale,Cal_image,Neighbors='4',Elvevation_model='weighted_avg_elevation')
            pbar.set_description('完成Points_WithH_Angle')

            Points_WithH_Angle, unionFilter_indices = Main_CalMaxPoints(Points_WithH_Angle,Orbit)
            distortion_features = Main_CalDistortion(Points_WithH_Angle, unionFilter_indices)
            pbar.set_description('完成distortion_features')

            Distortion = ee.Image().paint(distortion_features, 'values').reproject(crs=Projection, scale=Prj_scale).clip(AOI).toInt8()
            DataIO.Geemap_export(f'{i:06d}'+'_' +'Distortion'+ Orbit+ '.tif',Distortion,region=AOI,scale=Prj_scale,rename_image=False)

            First_derivative = ee.Image().paint(distortion_features, 'first_derivative_max').reproject(crs=Projection, scale=Prj_scale).clip(AOI).toInt8()
            DataIO.Geemap_export(f'{i:06d}'+'_'+'First_derivative'+ Orbit+ '.tif',First_derivative,region=AOI,scale=Prj_scale,rename_image=False)

        except:
            Wrong_dataIndex.append(i)
            # 记录错误信息
            with open('log.txt', 'a') as f: 
                f.write('Wrong index = {}\n'.format(i))
                f.write(traceback.format_exc())
                f.write('\n')
            print('错误已记录到log.txt')